# IMPORTING LIBRARIES

In [84]:
import sys
import pandas as pd
import requests
!conda install -c conda bs4 --yes
from bs4 import BeautifulSoup

Solving environment: done

# All requested packages already installed.



In [85]:
from bs4 import BeautifulSoup #importing library for web scraping from WIKIPEDIA

In [86]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'#url
response=requests.get(url)  #sending request to url
soup = BeautifulSoup(response.text,'html.parser') # getting the html code for the page

In [87]:
table=soup.find('table',{'class':'wikitable sortable'}).tbody #extracting required table out the webpage

In [88]:
rows=table.find_all('tr')                                               #getting rows
columns=[v.text.replace('\n','') for v in rows[0].find_all('th')]       #getting column_headers

In [165]:
df1 = pd.DataFrame(columns = columns)        #Converting to  Pandas Dataframe
df1.head()

,Postal Code,Borough,Neighborhood


In [166]:
for i in range(1,len(rows)):                                                           #adding values to columns
    tds=rows[i].find_all('td')
    values=[td.text.replace('\n','') for td in tds]
    df1=df1.append(pd.Series(values,index=columns),ignore_index=True)
#df.to_csv(r'C:\Users\Rahul singh\Downloads\COURSERA CERTIFICATE'+ '\\wikipedia.csv',index=False) #saving as csv
df_fil=df1[df1.Borough !='Not assigned'] # creating a df without BOUROUGH = 'Not assigned label'
#for entry in df_fil.values:
 #   if entry[2]=='':
  #      entry[2]=entry[1]
   # else:
    #    continue
    


    


In [167]:
#df_fil.to_csv(r'C:\Users\Rahul singh\Downloads\COURSERA CERTIFICATE'+'\\filtered.csv',index=False)
df_fil.reset_index(drop=True,inplace=True)

# FILTERED DATAFRAME

In [168]:
df_fil

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Business reply mail Processing Centre
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [169]:
print("The shape of the dataframe is:",df_fil.shape)  #SHAPE OF THE DATAFRAME

The shape of the dataframe is: (103, 3)


!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim

geolocator=Nominatim(user_agent='toronto_explorer')
spatial_loc_= pd.DataFrame(columns=['LATITUDE','LONGITUDE','ADDRESS'])
for code in df_fil['Postal Code']:
    try:
                     address= "{},Toronto,Ontario".format(code)
                     location= geolocator.geocode(address)
                     lat=location.latitude
                     long=location.longitude
                     spatial_loc.append(lat,long,address)
    except:
        pass

In [170]:
#spatial_loc.head()

In [171]:
!wget -q -o "spatial_coord.csv" http://cocl.us/Geospatial_data
print("Done")

Done


## READING GEOSPACIAL_DATA

In [172]:
spatial_df=pd.read_csv('Geospatial_data')

spatial_dataframe = pd.DataFrame(spatial_df)

In [174]:
complete_df=df_fil.merge(spatial_dataframe,left_on='Postal Code',right_on='Postal Code') #JOINING LATITUDE AND LONGITUDE

In [194]:
complete_df.sort_values(by=['Postal Code'])

,Postal Code,Borough,Neighborhood,Latitude,Longitude
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
18,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
22,M1G,Scarborough,Woburn,43.770992,-79.216917
26,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
64,M9N,York,Weston,43.706876,-79.518188
70,M9P,Etobicoke,Westmount,43.696319,-79.532242
77,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
89,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437


In [190]:
 Value_Target = complete_df.loc[complete_df['Postal Code']== 'M1B']['Latitude'].values